## Build agentic applications using Strands Agents open source SDK by AWS

### Set up AWS environment
- If you are running this notebook in SageMaker Notebook instance, you need to give SageMaker permission to access Bedrock.
- If you are running it in your local environment, follow the steps below to set up proper permission to access your AWS resouces, espeically Bedrock:

Strands supports many different model providers. By default, agents use the Amazon Bedrock model provider with the Claude 3.7 model.

To use the examples in this guide, you'll need to configure your environment with AWS credentials that have permissions to invoke the Claude 3.7 model. You can set up your credentials in several ways:

    Environment variables: Set AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, and optionally AWS_SESSION_TOKEN
    AWS credentials file: Configure credentials using aws configure CLI command
    IAM roles: If running on AWS services like EC2, ECS, or Lambda, use IAM roles

Make sure your AWS credentials have the necessary permissions to access Amazon Bedrock and invoke the Claude 3.7 model. You'll need to enable model access in the Amazon Bedrock console following the AWS documentation here: https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html

### Install Strands Agents and other packages
AWS Strands Agents Reference: https://strandsagents.com/0.1.x/user-guide/quickstart/

In [ ]:
!pip install strands-agents
!pip install strands-agents-tools 
!pip install strands-agents-builder
!pip install pinecone cohere
!pip install -U duckduckgo_search
!pip install pinecone

In [5]:
from strands import Agent
from strands.models import BedrockModel
from duckduckgo_search import DDGS
from botocore.config import Config
import boto3
import botocore
from datetime import datetime, timedelta, date
import json
from pinecone import Pinecone

### 1. Test some built-in tool

In [6]:
# Simple calculator tool

from strands_tools import calculator

In [ ]:
agent = Agent(tools=[calculator])
agent("What is the square root of 1764")

### 2. Build custom tool using Python functions

In [8]:
from strands import tool

#### Tool 1: Web search engine

In [9]:
# Tool 1: Web search engine

@tool
def web_search_engine(query: str) -> dict:
    '''
    Searches the internet and retrieves content relevant to the input query
    '''
    # search_results = DDGS().text("Company where Andy Jassy is CEO", max_results=5)
    search_results = DDGS().text(query, max_results=5)
    
    return search_results

#### Tool 2: Word count

In [10]:
@tool
def word_count(text: str) -> int:
    """Count words in text.

    This docstring is used by the LLM to understand the tool's purpose.
    """
    return len(text.split())



#### Tool 3: Pinecone search

In [ ]:
pinecone_api_key = '' #Provide Pinecone API key

index_name = 'pinecone-cohere-workshop' #Provide Pinecone Index name that we created in the 'Create Pinecone Index' section
namespace = 'agentic-rag'


pc = Pinecone(api_key=pinecone_api_key)

index = pc.Index(index_name)
print(index.describe_index_stats())

In [13]:
# Initialize Bedrock
config = Config(connect_timeout=5, read_timeout=60, retries={"total_max_attempts": 20, "mode": "adaptive"})
region = 'us-west-2'

bedrock = boto3.client(
                service_name='bedrock-runtime',
                region_name=region,
                endpoint_url=f'https://bedrock-runtime.{region}.amazonaws.com',
                                    config=config)

In [14]:
# Cohere Embedding model in Bedrock

def cohere_embed_query(docs: str, input_type: str) -> list[float]:
    """
    Generate text embedding by using the Cohere Embed model.
    Args:
        docs: string of text to embed.
        input_type: select between [search_document, search_query, classification, clustering, image]
    Returns:
        dict: embeddings in float type.
    """

    body = json.dumps({
        "texts": [docs],
        "input_type": input_type,
        "embedding_types": ["float"]

    })
    
    model_id = 'cohere.embed-english-v3'
    accept = 'application/json' 
    content_type = 'application/json'
    
    # Invoke model 
    response = bedrock.invoke_model(
        body=body, 
        modelId=model_id, 
        accept=accept, 
        contentType=content_type
    )
    
    # Parse response
    response_body = json.loads(response['body'].read())
    embedding = response_body.get('embeddings')['float'][0]
    
    return embedding

In [15]:
# Tool 3: Query Pinecone DB

@tool
def query_pinecone_db(query: str) -> list[dict]:
    '''
    Searches Pinecone vector database index to retrieve documents relevant to the input query. The index contain financial statement data related to Company Compaq
    '''
    
    query_embedding = cohere_embed_query(query, input_type="search_query")

    results = index.query(
        namespace=namespace,
        vector=query_embedding,
        top_k=20,
        include_metadata=True,
    )
    query_results = [{"chunk_text": rec['metadata']['chunk_text']} for rec in results['matches']]
    # Sample results would look like below
    #search_results = [
    #    {"chunk_text": "Amazon revenue for 2025 is $3 Billion"},
    #    {"chunk_text": "Amazon revenue for 2024 is $2 Billion"},
    #    {"chunk_text": "Amazon revenue for 2024 is $1 Billion"},
    #]
    return query_results


In [16]:
agent = Agent(tools=[web_search_engine,query_pinecone_db, word_count])

In [ ]:
agent = Agent(tools=[web_search_engine,query_pinecone_db, word_count])
response = agent("what's the revenue of the company Compaq?")

### Strands defaults to the Bedrock model provider using Claude 3.7 Sonnet. However, we can change the LLM to others.

In [18]:
# Switch to Cohere LLM model

bedrock_model = BedrockModel(
  model_id="cohere.command-r-plus-v1:0",
  region_name = "us-east-1",
  temperature=0.3,
)

agent = Agent(tools=[web_search_engine,query_pinecone_db, word_count], model=bedrock_model)

In [ ]:
response = agent("what's the revenue of the company Compaq?")